In [47]:
import polars as pl
from sklearn.decomposition import PCA

df = pl.read_csv("../Data/Titanic/train.csv")

In [48]:
def standardize(column):
    return (column - column.mean()) / column.std()

df = df.with_columns(
    (pl.col("Parch") + pl.col("SibSp") + 1).alias("FamilySize"),
    pl.col("Fare").fill_null(pl.col("Fare").median()).alias("Fare"),
    pl.col("Age").fill_null(pl.col("Age").median()).alias("Age"),
    pl.when(pl.col("Age") < 16).then(1).otherwise(0).alias("IsChild"),
    pl.when(pl.col("Cabin").is_not_null()).then(1).otherwise(0).alias("HasCabinRegistered"),
    ).with_columns(
        pl.col("Fare").log1p().alias("Fare"),
        pl.when((pl.col("FamilySize") >= 2) & (pl.col("FamilySize") <= 4)).then(1).otherwise(0).alias("HasSmallFamily"),
        ).to_dummies([
            "Sex", 
            "Embarked"
            ]).drop([
                "Sex_male",
                "Parch",
                "SibSp",
                "Name",
                "Ticket",
                "Cabin",
                "Embarked_S",
                # "Embarked_Q",
                "Embarked_null",
                # "FamilySize"
                ], 
                strict=False
                ).with_columns(
                    standardize(pl.col("Fare")).alias("Fare"),
                    standardize(pl.col("Age")).alias("Age")
                )

# pca = PCA().fit_transform(df["Pclass", "HasCabinRegistered", "Fare"])

# df = df.with_columns(
#     pl.lit(pca[:, 0]).alias("PC1"),
#     pl.lit(pca[:, 1]).alias("PC2"),
#     pl.lit(pca[:, 2]).alias("PC3")
# ).drop("Pclass", "HasCabinRegistered", "Fare")

In [49]:
df.describe()

statistic,PassengerId,Survived,Pclass,Sex_female,Age,Fare,Embarked_C,Embarked_Q,FamilySize,IsChild,HasCabinRegistered,HasSmallFamily
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",891.0,891.0,891.0,891.0,891.0,891.0,891.0,891.0,891.0,891.0,891.0,891.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",446.0,0.383838,2.308642,0.352413,2.5519e-16,-3.1899e-16,0.188552,0.08642,1.904602,0.093154,0.228956,0.327722
"""std""",257.353842,0.486592,0.836071,0.47799,1.0,1.0,0.391372,0.281141,1.613459,0.290811,0.420397,0.469646
"""min""",1.0,0.0,1.0,0.0,-2.222908,-3.056861,0.0,0.0,1.0,0.0,0.0,0.0
"""25%""",224.0,0.0,2.0,0.0,-0.565419,-0.798092,0.0,0.0,1.0,0.0,0.0,0.0
"""50%""",446.0,0.0,3.0,0.0,-0.104579,-0.231531,0.0,0.0,1.0,0.0,0.0,0.0
"""75%""",669.0,1.0,3.0,1.0,0.433068,0.519572,0.0,0.0,2.0,0.0,0.0,1.0
"""max""",891.0,1.0,3.0,1.0,3.88937,3.383393,1.0,1.0,11.0,1.0,1.0,1.0


In [50]:
df.write_csv("../Data/Titanic/advancedTrain.csv")

df.head()

PassengerId,Survived,Pclass,Sex_female,Age,Fare,Embarked_C,Embarked_Q,FamilySize,IsChild,HasCabinRegistered,HasSmallFamily
i64,i64,i64,u8,f64,f64,u8,u8,i64,i32,i32,i32
1,0,3,0,-0.565419,-0.879247,0,0,2,0,0,1
2,1,1,1,0.663488,1.360456,1,0,2,0,1,1
3,1,3,1,-0.258192,-0.798092,0,0,1,0,0,0
4,1,1,1,0.433068,1.061442,0,0,2,0,1,1
5,0,3,0,0.433068,-0.783739,0,0,1,0,0,0
